horse_idをinputして5代血統データをoutputするプログラム

# 準備
## インポート

In [1]:
# スクレイピングに関するクラス
import requests
from bs4 import BeautifulSoup

import urllib.parse

# タイムバーに関するクラス
from tqdm.notebook import tqdm

# 時間制御に関するクラス
import time

# その他便利クラス
import math
import numpy as np
import pandas as pd
import os
from io import StringIO
import re

## 定数

In [2]:
# ヘッダーの設定に用いる定数
HEADERS_DIC = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.97 Safari/537.36"
}

## 便利関数

In [3]:
class Tool:
    @staticmethod
    def arrangePed(ped_str):
        """
        血統表に存在する文字から必要な文字のみを抽出する関数
        例: Northern Dancer 1961 鹿毛 -> Northern Dancer
        例: Kingman (英) 2011 鹿毛 [血統][産駒] Danzig系 -> Kingman
        """
        def deleteExtraWord(ped_string):
            # 前提 : 馬の名前にアラビア数字が入らないものとする
            # アラビア数字が混在している場合、'馬名 誕生年 その他情報'という文字列と考える
            if (type(ped_string) != type('str')):
                return np.nan

            # アラビア数字がない場合はそのまま返す
            if not(bool(re.search(r'\d', ped_string))):
                return ped_string

            organize_string  = re.split('[0-9]', ped_string)[0][:-1]
            if ('(' in organize_string):
                if (organize_string.split('(')[0][-1] == ' '):
                    return organize_string.split('(')[0][:-1]
                return organize_string.split('(')[0]
            else:
                return organize_string 
        
        """ex. 'マンファス Manfath'という文字列を'マンファス'と出力する"""
        p = re.compile('[\u30A1-\u30FF]+')
        arrange_ped_str = deleteExtraWord(ped_str)
        if p.search(arrange_ped_str):
            p2 = re.compile('[I\u2160-\u217F]+')
            if (p2.search(arrange_ped_str)):
                return re.compile('[I\u30A1-\u30FF\u2160-\u217F]+').findall(arrange_ped_str)[0]
            return p.findall(arrange_ped_str)[0]
        else:
            return arrange_ped_str
        
    def delData(file_path, row_id_list):
        df = pd.read_pickle(file_path)
        df.drop(row_id_list).to_pickle(file_path)
        print('delete data.')

In [4]:
Tool.arrangePed('Northern Dancer 1961 鹿毛')

'Northern Dancer'

In [5]:
Tool.arrangePed('Kingman (英) 2011 鹿毛 [血統][産駒] Danzig系')

'Kingman'

In [6]:
Tool.arrangePed('Irish Lass')

'Irish Lass'

# 血統表抽出

In [48]:
#血統データを処理するクラス
class Peds:
    @staticmethod
    def getData(horse_id_list, get_id = False):
        """
        血統データをスクレイピングする関数

        Parameters:
        ----------
        horse_id_list : list
            馬IDのリスト

        Returns:
        ----------
        peds_df : pandas.DataFrame
            全血統データをまとめてDataFrame型にしたもの
        """
        peds_dict = {}
        session = requests.Session()
        
        for horse_id in tqdm(horse_id_list):
            time.sleep(1)
            try:
                url = "https://db.netkeiba.com/horse/ped/" + horse_id
                
                # horse_idの馬名を取得
                response = session.get(url, headers=HEADERS_DIC)
                response.encoding = response.apparent_encoding
                soup = BeautifulSoup(response.text, "html.parser")
                
                # 馬名取得
                horse_name = soup.find("div", attrs={"id": "db_main_box"}).find("h1").get_text()
                
                # 5代血統表作成
                df = pd.read_html(StringIO(response.text))[0]

                #重複を削除して1列のSeries型データに直す
                generations = {}
                for i in reversed(range(5)):
                    generations[i] = df[i]
                    df.drop([i], axis=1, inplace=True)
                    df = df.drop_duplicates()
                
                ped = pd.concat([pd.Series([horse_name])] + [generations[i] for i in range(5)]).rename(horse_id)
                peds_dict[horse_id] = ped.reset_index(drop=True)

            except IndexError:
                continue
            except Exception as e:
                print(horse_id+':', e)
                continue
            except:
                break
                
        
        # これ以降はlen(peds_dict) > 0
        #列名をpeds_0, ..., peds_62にする
        peds_df = pd.concat([peds_dict[key] for key in peds_dict], axis=1).T.add_prefix('peds_')
        
        # 血統表内の文字列を整形する
        peds_df = pd.concat([peds_df['peds_0'], peds_df[['peds_'+str(i) for i in range(1, 62+1)]].map(Tool.arrangePed)], axis=1)
        return peds_df
    
    def getId(horse_id_list):
        """
        血統データをスクレイピングする関数

        Parameters:
        ----------
        horse_id_list : list
            馬IDのリスト

        Returns:
        ----------
        peds_df : pandas.DataFrame
            全血統データをまとめてDataFrame型にしたもの
        """
        peds_id_dict = {}
        session = requests.Session()
        
        for horse_id in tqdm(horse_id_list):
            time.sleep(1)
            try:
                url = "https://db.netkeiba.com/horse/ped/" + horse_id
                
                ped_horse_id_list = [0]*63 # 空リストを用意
                ped_horse_id_list[0] = horse_id # 1つ目の要素としてhorse_idを格納
                
                # ped horse idを並び替える
                label = 1 # ped_horse_idを格納する場所のラベル
                is_house_serial_number = False # 連番ラベルに格納したか
                
                response = session.get(url, headers=HEADERS_DIC)
                response.encoding = response.apparent_encoding
                soup = BeautifulSoup(response.text, "html.parser")
                
                horse_a_list = soup.find("table", attrs={"class": "blood_table"}).find_all(
                    "a", attrs={"href": re.compile(r".*?/db.netkeiba.com/horse/\d+")}
                )
                for a in horse_a_list:
                    ped_horse_id = a["href"].split('/')[-2]
                    ped_horse_id_list[label] = ped_horse_id
                    
                    if (is_house_serial_number):
                        is_house_serial_number = False
                        if (label % 4 == 0):
                            label = int(label / 2)
                            continue
                        if (label % 8 == 2):
                            label = int((label-2) / 4)
                            continue
                        if (label % 16 == 6):
                            label = int((label-6) / 8)
                            continue
                        label = int((label-14) / 16)
                        continue
                        
                    if (label*2 < 61):
                        label = label * 2 + 1
                        is_house_serial_number = False
                        continue
                    else:
                        label = label + 1
                        is_house_serial_number = True
                
                peds_id_dict[horse_id] = pd.DataFrame(ped_horse_id_list)
               
            except IndexError:
                print(horse_id+':', e)
                continue
            except Exception as e:
                print(horse_id+':', e)
                continue
            except:
                break
        
        # これ以降はlen(peds_dict) > 0
        #列名をpeds_0, ..., peds_62にする
        peds_id_df = pd.concat([peds_id_dict[key] for key in peds_id_dict], axis=1).T.add_prefix('peds_')
        
        return peds_id_df.set_index('peds_0', drop=True)
    
    def scrape(horse_id_list):
        return Peds.getData(horse_id_list, get_id=True)

In [19]:
Peds.scrape(['2018103559'])

  0%|          | 0/1 [00:00<?, ?it/s]

,peds_0,peds_1,peds_2,peds_3,peds_4,peds_5,peds_6,peds_7,peds_8,peds_9,...,peds_53,peds_54,peds_55,peds_56,peds_57,peds_58,peds_59,peds_60,peds_61,peds_62
2018103559,タイトルホルダー,ドゥラメンテ,メーヴェ,キングカメハメハ,アドマイヤグルーヴ,Motivator,Top Table,Kingmambo,マンファス,サンデーサイレンス,...,Sharpen Up,Uncommitted,Never Bend,Milan Mill,ハーディカヌート,Grand Cross,Northern Dancer,What a Treat,Lorenzaccio,Courtessa


In [50]:
Peds.getId(['2018103559'])

  0%|          | 0/1 [00:00<?, ?it/s]

,peds_1,peds_2,peds_3,peds_4,peds_5,peds_6,peds_7,peds_8,peds_9,peds_10,...,peds_53,peds_54,peds_55,peds_56,peds_57,peds_58,peds_59,peds_60,peds_61,peds_62
peds_0,,,,,,,,,,,,,,,,,,,,,
2018103559,2012104511,2008110046,2001103460,2000107542,000a010c91,000a0116ec,000a001d7e,000a00680e,000a00033a,1993109154,...,000a001677,000a009476,000a000e03,000a007ccf,000a00043a,000a0080dc,000a000e04,000a007e1b,000a000f66,000a0069db


In [51]:
Peds.scrape(['2013106101', '2017101835'])

  0%|          | 0/2 [00:00<?, ?it/s]

,peds_0,peds_1,peds_2,peds_3,peds_4,peds_5,peds_6,peds_7,peds_8,peds_9,...,peds_53,peds_54,peds_55,peds_56,peds_57,peds_58,peds_59,peds_60,peds_61,peds_62
2013106101,サトノダイヤモンド,ディープインパクト,マルペンサ,サンデーサイレンス,ウインドインハーヘア,Orpen,Marsella,Halo,Wishing Well,Alzao,...,Hoist the Flag,Natalma,Hail to Reason,Cosmah,Northern Dancer,Sea Saga,Buckpasser,Smart Deb,Tatan,Ravana
2017101835,コントレイル,ディープインパクト,ロードクロサイト,サンデーサイレンス,ウインドインハーヘア,Unbridled's Song,Folklore,Halo,Wishing Well,Alzao,...,Lucky Mel,Incantation,Relaunch,テイズリー,Seattle Song,Lonely Dancer,Storm Bird,Terlingua,Fappiano,バジー


In [52]:
Peds.getId(['2013106101', '2017101835'])

  0%|          | 0/2 [00:00<?, ?it/s]

,peds_1,peds_2,peds_3,peds_4,peds_5,peds_6,peds_7,peds_8,peds_9,peds_10,...,peds_53,peds_54,peds_55,peds_56,peds_57,peds_58,peds_59,peds_60,peds_61,peds_62
peds_0,,,,,,,,,,,,,,,,,,,,,
2013106101,2002100816,000a011cdc,000a00033a,000a0003a2,000a002326,000a011cb2,000a0012bf,000a008c1e,000a001cb4,000a00922c,...,000a0012be,000a00702e,000a000f2b,000a007459,000a000e04,000a009cdd,000a000e0e,000a00a3ec,000a00101f,000a011caf
2017101835,2002100816,2010110076,000a00033a,000a0003a2,000a002071,000a011d47,000a0012bf,000a008c1e,000a001cb4,000a00922c,...,000a000f67,000a00869f,000a001b45,000a000364,000a0017e6,000a00faee,000a0016d4,000a008e05,000a001702,000a000059
